# Proyecto Modelos I - UdeA
## Tercer Entrega

En este notebook se desarrolla la fase final del proyecto del curso “Modelos y Simulación de Sistemas I” de la Universidad de Antioquia, donde se realiza el preprocesado y se construye, entrena y evalúa un modelo de aprendizaje automático que permita predecir el rendimiento global de los estudiantes, en el contexto de la competencia “Pruebas Saber Pro Colombia”.

El propósito de esta entrega es implementar un flujo completo de modelado predictivo, iniciando desde la carga del dataset procesado y finalizando con la generación del archivo submission.csv requerido para la plataforma Kaggle.

<br>
---

# Antes de empezar
Es necesario importar y descargar los archivos requeridos desde la plataforma Kaggle. Para ello, se cargan primero las librerías necesarias en el notebook y, posteriormente, se descarga el archivo comprimido (.zip) que contiene el dataset principal (train.csv), el cual será utilizado en el proceso de exploración.

In [1]:
import os
import unicodedata
import re
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, classification_report
from xgboost import XGBClassifier


os.environ['KAGGLE_CONFIG_DIR'] = '.'
!chmod 600 ./kaggle.json
!kaggle competitions download -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia

  0% 0.00/29.9M [00:00<?, ?B/s]
100% 29.9M/29.9M [00:00<00:00, 1.03GB/s]


## Descomprimimos el archivo .zip

In [2]:
# Comando de linux para descomprimir sin mostrar nada en pantalla
!unzip udea*.zip > /dev/null

## Cargamos los datos a analizar en un dataframe y verificamos que si haya cargado correctamente

In [3]:
# Cargar datos
df = pd.read_csv("train.csv",encoding='utf-8')
df_test = pd.read_csv("test.csv",encoding='utf-8')


In [4]:
# Verificamos la carga exitosa de los datos
print("Número de filas y columnas:", df.shape)
print("\nVista previa:")
display(df.head())
# Resumen estadístico
df.describe(include='all')

Número de filas y columnas: (692500, 21)

Vista previa:


,ID,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,...,E_PRIVADO_LIBERTAD,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,F_TIENEINTERNET.1,F_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,...,N,No,Si,Si,Postgrado,medio-alto,0.322,0.208,0.310,0.267
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,...,N,No,Si,No,Técnica o tecnológica incompleta,bajo,0.311,0.215,0.292,0.264
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,...,N,No,No,Si,Secundaria (Bachillerato) completa,bajo,0.297,0.214,0.305,0.264
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,...,N,No,Si,Si,Secundaria (Bachillerato) completa,alto,0.485,0.172,0.252,0.190
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,...,N,No,Si,Si,Primaria completa,medio-bajo,0.316,0.232,0.285,0.294


,ID,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,...,E_PRIVADO_LIBERTAD,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,F_TIENEINTERNET.1,F_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
count,692500.000000,692500.000000,692500,692500,686213,661643,660363,665871,669322,652727,...,692500,686002,654397,665871,668836,692500,692500.000000,692500.000000,692500.000000,692500.000000
unique,NaN,NaN,948,31,8,5,7,2,12,2,...,2,2,2,2,12,4,NaN,NaN,NaN,NaN
top,NaN,NaN,DERECHO,BOGOTÁ,Entre 1 millón y menos de 2.5 millones,Más de 30 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Si,...,N,No,Si,Si,Secundaria (Bachillerato) completa,alto,NaN,NaN,NaN,NaN
freq,NaN,NaN,53244,282159,204048,249352,232671,592514,128289,563390,...,692466,382201,597670,592514,141744,175619,NaN,NaN,NaN,NaN
mean,494606.130576,20198.366679,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.268629,0.259996,0.262087,0.262903
std,285585.209455,10.535037,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.122130,0.093480,0.058862,0.067944
min,1.000000,20183.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000
25%,247324.750000,20195.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.203000,0.212000,0.254000,0.255000
50%,494564.500000,20195.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.240000,0.271000,0.276000,0.285000
75%,741782.500000,20203.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.314000,0.309000,0.293000,0.303000


## Limpieza de datos - Columnas que tengan texto

Se procede a limpiar las columnas que contienen texto, eliminando tildes, mayúsculas y caracteres especiales.

<b>Nota</b>: Omitimos la limpieza en la columna E_VALORMATRICULAUNIVERSIDAD ya que esta columna contiene datos con números separados por punto.


In [5]:
def limpiar_texto(texto):
    if pd.isna(texto):
        return texto
    # Convertir a string
    texto = str(texto)
    # Quitar tildes y acentos
    texto = unicodedata.normalize('NFKD', texto).encode('ascii', 'ignore').decode('utf-8')
    # Pasar a minúsculas
    texto = texto.lower()
    # Quitar caracteres no alfabéticos ni numéricos (dejando espacios)
    texto = re.sub(r'[^a-z0-9\s]', '', texto)
    # Quitar espacios extra
    texto = re.sub(r'\s+', ' ', texto).strip()
    return texto


# Detectar las columnas de tipo texto (object o string)
columnas_texto = df.select_dtypes(include=['object', 'string']).columns
columnas_texto = columnas_texto.drop("E_VALORMATRICULAUNIVERSIDAD")
columnas_texto_test = df_test.select_dtypes(include=['object', 'string']).columns
columnas_texto_test = columnas_texto_test.drop("E_VALORMATRICULAUNIVERSIDAD")

# Aplicar la limpieza
for col in columnas_texto:
  df[col] = df[col].apply(limpiar_texto)
for col in columnas_texto_test:
  df_test[col] = df_test[col].apply(limpiar_texto)

print(f"Columnas limpiadas: {list(columnas_texto)}")

Columnas limpiadas: ['E_PRGM_ACADEMICO', 'E_PRGM_DEPARTAMENTO', 'E_HORASSEMANATRABAJA', 'F_ESTRATOVIVIENDA', 'F_TIENEINTERNET', 'F_EDUCACIONPADRE', 'F_TIENELAVADORA', 'F_TIENEAUTOMOVIL', 'E_PRIVADO_LIBERTAD', 'E_PAGOMATRICULAPROPIO', 'F_TIENECOMPUTADOR', 'F_TIENEINTERNET.1', 'F_EDUCACIONMADRE', 'RENDIMIENTO_GLOBAL']


## Creamos un diccionario con categorías para agrupar todos nuestros programas

In [6]:
# --- Diccionario de áreas por palabras clave ---
areas_dict = {
    "ingenieria": "Ingenierías",
    "industrial": "Ingenierías",
    "sistemas": "Ingenierías",
    "informatica": "Ingenierías",
    "tecnologia": "Ingenierías",
    "quimica": "Ciencias Naturales",
    "fisica": "Ciencias Naturales",
    "biologia": "Ciencias Naturales",
    "ambiental": "Ciencias Naturales",
    "comunicacion": "Ciencias Sociales",
    "periodismo": "Ciencias Sociales",
    "sociologia": "Ciencias Sociales",
    "psicologia": "Ciencias Sociales",
    "derecho": "Ciencias Sociales",
    "economia": "Ciencias Económicas",
    "contaduria": "Ciencias Económicas",
    "administracion": "Ciencias Económicas",
    "mercadeo": "Ciencias Económicas",
    "medicina": "Ciencias de la Salud",
    "enfermeria": "Ciencias de la Salud",
    "odontologia": "Ciencias de la Salud",
    "fisioterapia": "Ciencias de la Salud",
    "licenciatura": "Educación",
    "pedagogia": "Educación",
    "educacion": "Educación",
    "musica": "Artes y Humanidades",
    "arte": "Artes y Humanidades",
    "diseno": "Artes y Humanidades",
    "filosofia": "Artes y Humanidades",
    "historia": "Artes y Humanidades",
    "arquitectura": "Arquitectura y Diseño"
}

# --- Función para clasificar según palabras clave ---
def clasificar_area(programa):
    for palabra, area in areas_dict.items():
        if palabra in programa:
            return area
    return "Otros"

# Crear nueva columna
df["AREA_ACADEMICA"] = df["E_PRGM_ACADEMICO"].apply(clasificar_area)
df_test["AREA_ACADEMICA"] = df_test["E_PRGM_ACADEMICO"].apply(clasificar_area)

print(df["AREA_ACADEMICA"].value_counts())

AREA_ACADEMICA
Ciencias Económicas      192545
Ingenierías              154272
Ciencias Sociales        123713
Otros                     75629
Educación                 51950
Ciencias de la Salud      46958
Ciencias Naturales        22325
Artes y Humanidades       13382
Arquitectura y Diseño     11726
Name: count, dtype: int64


## Verificación e imputación de los valores nulos

In [7]:
# Verificar valores nulos
valores_nulos = df.isnull().sum()
print("Valores nulos por columna:\n", valores_nulos[valores_nulos > 0])

# --- Imputación de valores faltantes ---

# Imputar valores numéricos con la mediana
columnas_numericas = df.select_dtypes(include=['number']).columns
columnas_numericas_test = df_test.select_dtypes(include=['number']).columns
df[columnas_numericas] = df[columnas_numericas].fillna(df[columnas_numericas].median())
df_test[columnas_numericas_test] = df_test[columnas_numericas_test].fillna(df_test[columnas_numericas_test].median())

# Imputar valores categóricos (texto) con la etiqueta 'sin_dato'
columnas_categoricas = df.select_dtypes(include=['object', 'bool']).columns
columnas_categoricas_test = df_test.select_dtypes(include=['object', 'bool']).columns
df[columnas_categoricas] = df[columnas_categoricas].fillna('sin_dato')
df_test[columnas_categoricas_test] = df_test[columnas_categoricas_test].fillna('sin_dato')

# --- Verificación final ---
print("\nValores nulos después de imputar:")
print(df.isnull().sum()[df.isnull().sum() > 0])



Valores nulos por columna:
 E_VALORMATRICULAUNIVERSIDAD     6287
E_HORASSEMANATRABAJA           30857
F_ESTRATOVIVIENDA              32137
F_TIENEINTERNET                26629
F_EDUCACIONPADRE               23178
F_TIENELAVADORA                39773
F_TIENEAUTOMOVIL               43623
E_PAGOMATRICULAPROPIO           6498
F_TIENECOMPUTADOR              38103
F_TIENEINTERNET.1              26629
F_EDUCACIONMADRE               23664
dtype: int64

Valores nulos después de imputar:
Series([], dtype: int64)


## Codificación de variables categóricas

Las variables categóricas deben transformarse a formato numérico para ser interpretadas por nuestro modelo en futuras entregas.

In [8]:
# Seleccionar columnas categóricas relevantes
columnas_categoricas = [
    'AREA_ACADEMICA',
    'E_PRGM_DEPARTAMENTO', 'E_VALORMATRICULAUNIVERSIDAD',
    'E_HORASSEMANATRABAJA', 'F_ESTRATOVIVIENDA', 'F_TIENEINTERNET',
    'F_EDUCACIONPADRE', 'F_TIENELAVADORA', 'F_TIENEAUTOMOVIL',
    'E_PRIVADO_LIBERTAD', 'E_PAGOMATRICULAPROPIO', 'F_TIENECOMPUTADOR',
    'F_TIENEINTERNET.1', 'F_EDUCACIONMADRE'
]

mapa_rendimiento = {
    "bajo": 0,
    "mediobajo": 1,
    "medioalto": 2,
    "alto": 3
}


# Aplicar One-Hot Encoding
df_encoded = pd.get_dummies(df, columns=columnas_categoricas, drop_first=True)
df.drop(columns=["E_PRGM_ACADEMICO"], inplace=True)
df_encoded.drop(columns=["E_PRGM_ACADEMICO"], inplace=True)
df_encoded["RENDIMIENTO_GLOBAL"] = df_encoded["RENDIMIENTO_GLOBAL"].map(mapa_rendimiento)
df_encoded_test = pd.get_dummies(df_test, columns=columnas_categoricas, drop_first=True)
df_test.drop(columns=["E_PRGM_ACADEMICO"], inplace=True)
df_encoded_test.drop(columns=["E_PRGM_ACADEMICO"], inplace=True)
print("Dimensiones antes de codificación:", df.shape)
print("Dimensiones tras codificación:", df_encoded.shape)
# Convertir booleanos a binarios
df_encoded = df_encoded.astype({col: int for col in df_encoded.select_dtypes(include='bool').columns})
df_encoded_test = df_encoded_test.astype({col: int for col in df_encoded_test.select_dtypes(include='bool').columns})

df_encoded.head()



Dimensiones antes de codificación: (692500, 21)
Dimensiones tras codificación: (692500, 102)


,ID,PERIODO_ACADEMICO,RENDIMIENTO_GLOBAL,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4,AREA_ACADEMICA_Artes y Humanidades,AREA_ACADEMICA_Ciencias Económicas,AREA_ACADEMICA_Ciencias Naturales,...,F_EDUCACIONMADRE_no aplica,F_EDUCACIONMADRE_no sabe,F_EDUCACIONMADRE_postgrado,F_EDUCACIONMADRE_primaria completa,F_EDUCACIONMADRE_primaria incompleta,F_EDUCACIONMADRE_secundaria bachillerato completa,F_EDUCACIONMADRE_secundaria bachillerato incompleta,F_EDUCACIONMADRE_sin_dato,F_EDUCACIONMADRE_tecnica o tecnologica completa,F_EDUCACIONMADRE_tecnica o tecnologica incompleta
0,904256,20212,2,0.322,0.208,0.310,0.267,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,645256,20212,0,0.311,0.215,0.292,0.264,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,308367,20203,0,0.297,0.214,0.305,0.264,0,1,0,...,0,0,0,0,0,1,0,0,0,0
3,470353,20195,3,0.485,0.172,0.252,0.190,0,1,0,...,0,0,0,0,0,1,0,0,0,0
4,989032,20212,1,0.316,0.232,0.285,0.294,0,0,0,...,0,0,0,1,0,0,0,0,0,0


## Guardamos nuestro nuevo Dataset preprocesado

In [9]:
# Guardar el dataset preprocesado
df_encoded.to_csv("train_preprocesado.csv", index=False)
df_encoded_test.to_csv("test_preprocesado.csv", index=False)
print("Archivo preprocesado guardado correctamente como 'train_preprocesado.csv' y 'test_preprocesado.csv'")


Archivo preprocesado guardado correctamente como 'train_preprocesado.csv' y 'test_preprocesado.csv'


# Modelo

Ahora empezamos a leer el archivo preprocesado anteriormente, para entrenar nuestro modelo.

In [10]:
df = pd.read_csv("train_preprocesado.csv")

# Target y features
X = df.drop(columns=["RENDIMIENTO_GLOBAL"])
y = df["RENDIMIENTO_GLOBAL"]
df.head()


,ID,PERIODO_ACADEMICO,RENDIMIENTO_GLOBAL,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4,AREA_ACADEMICA_Artes y Humanidades,AREA_ACADEMICA_Ciencias Económicas,AREA_ACADEMICA_Ciencias Naturales,...,F_EDUCACIONMADRE_no aplica,F_EDUCACIONMADRE_no sabe,F_EDUCACIONMADRE_postgrado,F_EDUCACIONMADRE_primaria completa,F_EDUCACIONMADRE_primaria incompleta,F_EDUCACIONMADRE_secundaria bachillerato completa,F_EDUCACIONMADRE_secundaria bachillerato incompleta,F_EDUCACIONMADRE_sin_dato,F_EDUCACIONMADRE_tecnica o tecnologica completa,F_EDUCACIONMADRE_tecnica o tecnologica incompleta
0,904256,20212,2,0.322,0.208,0.310,0.267,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,645256,20212,0,0.311,0.215,0.292,0.264,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,308367,20203,0,0.297,0.214,0.305,0.264,0,1,0,...,0,0,0,0,0,1,0,0,0,0
3,470353,20195,3,0.485,0.172,0.252,0.190,0,1,0,...,0,0,0,0,0,1,0,0,0,0
4,989032,20212,1,0.316,0.232,0.285,0.294,0,0,0,...,0,0,0,1,0,0,0,0,0,0


# Separamos las caracterísitcas y entrenamos el modelo.

Usamos un XGBoost sencillo.

In [11]:
X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [12]:
model = XGBClassifier(
    n_estimators=300,
    max_depth=6,
    learning_rate=0.05,
    subsample=0.9,
    colsample_bytree=0.9,
    eval_metric="mlogloss"
)

model.fit(X_train, y_train)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.9, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.05, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=300, n_jobs=None,
              num_parallel_tree=None, ...)

# Verificamos exactitud de nuestro modelo.

In [13]:
y_pred = model.predict(X_valid)

print("Accuracy:", accuracy_score(y_valid, y_pred))
print("F1 macro:", f1_score(y_valid, y_pred, average="macro"))
print("\nClassification report:\n", classification_report(y_valid, y_pred))


Accuracy: 0.418
F1 macro: 0.4031982226372957

Classification report:
               precision    recall  f1-score   support

           0       0.43      0.56      0.49     34597
           1       0.33      0.27      0.30     34455
           2       0.32      0.23      0.27     34324
           3       0.52      0.61      0.56     35124

    accuracy                           0.42    138500
   macro avg       0.40      0.42      0.40    138500
weighted avg       0.40      0.42      0.40    138500



# Ahora abrimos el archivo de prueba test.csv y ejecutamos nuestro modelo.

In [ ]:
test = pd.read_csv("test_preprocesado.csv")

test_ids = test["ID"]
pred_test = model.predict(test)


# Le damos formato categórico nuevamente a los valores de la columna de Rendimiento.

In [ ]:
mapping = {
    0: "bajo",
    1: "medio-bajo",
    2: "medio-alto",
    3: "alto"
}

pred_text = pd.Series(pred_test).map(mapping)


# Creamos nuestro archivo de submission.

In [ ]:
submission = pd.DataFrame({
    "ID": test_ids,
    "RENDIMIENTO_GLOBAL": pred_text
})

submission.to_csv("submission.csv", index=False)

print("Archivo submission.csv generado correctamente.")


Archivo submission.csv generado correctamente.


## Conclusión

En este notebook se completó la etapa de predicción de los datos.

Este proceso refleja la importancia de combinar un buen preprocesamiento, una correcta elección de algoritmos y una validación rigurosa para construir modelos robustos.
